# Face to Emoji (CS445 Course Project)

In [1]:
import os
from tqdm import tqdm
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms

## Parameters

In [2]:
# model parameters
NUM_EMOJI = 7

# training parameters
BATCH_SIZE = 32
NUM_EPOCHS = 20
LEARNING_RATE = 1e-3
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# path parameters
DATA_PATH = './data'
WEIGHTS_PATH = './weights'
RESTORE = False

## Data

In [3]:
# Load ImageNette dataset
img_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((224, 224))]
)
train_set = datasets.ImageFolder(root="./data/CK+48/train", transform=img_transform)
val_set = datasets.ImageFolder(root="./data/CK+48/val", transform=img_transform)

train_loader = torch.utils.data.DataLoader(train_set,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(train_set,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)

## Motify model

In [4]:
pipeline = models.resnet18(pretrained=True)
num_ftrs = pipeline.fc.in_features
pipeline.fc = nn.Linear(num_ftrs, NUM_EMOJI)
pipeline = pipeline.to(device)

## Training

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pipeline.parameters(), lr=LEARNING_RATE, momentum=0.9)

In [6]:
for epoch in tqdm(range(NUM_EPOCHS)):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = pipeline(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
#         running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
#             running_loss = 0.0

print('Finished Training')

100%|█████████████████████████████████████████████████████████████████████████| 20/20 [54:51<00:00, 164.57s/it]

Finished Training


## Save model

In [7]:
WEIGHTS_PATH = "./weights/resnet18_weights.pth"

In [8]:
torch.save(pipeline.state_dict(), WEIGHTS_PATH)

## Load model

In [ ]:
pipeline = models.resnet18(pretrained=True)
num_ftrs = pipeline.fc.in_features
pipeline.fc = nn.Linear(num_ftrs, NUM_EMOJI)
pipeline = pipeline.to(device)

model.load_state_dict(torch.load(WEIGHTS_PATH))
model.eval()

## test